In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import interpolate
from mpl_toolkits.mplot3d import Axes3D
import matplotlib
matplotlib.style.use('ggplot')
%matplotlib

Using matplotlib backend: Qt5Agg


In [191]:
# let's define the different wavelengths for the different files.
# I use this in order to open the right files
folder_path = '../data/adas/'
wavelengths = ['4101.2', '4339.9', '4860.6', '6561.9']

# try to read density and temperature from different files
n_e = pd.read_csv(folder_path + 'density_de.txt', delim_whitespace=True,
                 squeeze=True, header=None)
T_e = pd.read_csv(folder_path + 'temperature_de.txt', delim_whitespace=True,
                 squeeze=True, header=None)
n_e = n_e.T
T_e = T_e.T
n_e.columns = ['n_e']
T_e.columns = ['T_e']

In [192]:
# we should read the number of lines for every file, but I hope
# every file has the same lines number. In case this won't be true,
# this function has to be changed with range(0, 96) to range(0, num_lines)
df = pd.DataFrame()
for wavelength in wavelengths:
    
    file_path = folder_path + wavelength + '.txt'
    # read four dataframes because the files is splitted across 4 lines
    df0 = pd.read_csv(file_path, delim_whitespace=True,
                     skiprows=[x for x in range(0, 96) if (x+4)%4 != 0],
                     header=None)
    df1 = pd.read_csv(file_path, delim_whitespace=True,
                     skiprows=[x for x in range(0, 96) if (x+3)%4 != 0],
                     header=None)
    df2 = pd.read_csv(file_path, delim_whitespace=True,
                     skiprows=[x for x in range(0, 96) if (x+2)%4 != 0],
                     header=None)
    df3 = pd.read_csv(file_path, delim_whitespace=True,
                     skiprows=[x for x in range(0, 96) if (x+1)%4 != 0],
                     header=None)
    df_wavelength = pd.concat([df0, df1, df2, df3], axis=1, ignore_index=True)
    df_wavelength = df_wavelength.set_index(n_e['n_e'])
    df_wavelength.columns = T_e['T_e']
    df_wavelength['wavelength'] = wavelength
    df_wavelength['n_e'] = df_wavelength.index.values
    df_melt = pd.melt(df_wavelength, id_vars=['n_e', 'wavelength'], var_name='T_e',
                     value_name='X_e')
    df = df.append(df_melt)

In [193]:
a = df.groupby('wavelength', as_index=False)
# afaik h-alpha = 6561.9
# h-beta 4860.6
# h-gamma 4339.9
# h-delta 4101.2
# let's create a dataframe with the following columns:
# | wl1 | wl2 | X1 | X2 | Te | ne | ratio |
# the ration will be the respect to the following:
# ha/hb ---- ha/hg ----- ha/hd
ha = wavelengths[3]
hb = wavelengths[2]
hg = wavelengths[1]
hd = wavelengths[0]

df_ha = a.get_group(wavelengths[3])
df_hb = a.get_group(wavelengths[2])
df_hg = a.get_group(wavelengths[1])
df_hd = a.get_group(wavelengths[0])
# create a new dataframe for ratios
# merge df_ha with df_hb, df_ha with df_hg and so on
df_ratio = pd.DataFrame()
df_ratio = df_ratio.append(pd.merge(df_ha, df_hb, on=['n_e', 'T_e'], 
                                    suffixes=('_1', '_2')))
df_ratio = df_ratio.append(pd.merge(df_ha, df_hg, on=['n_e', 'T_e'], 
                                    suffixes=('_1', '_2')))
df_ratio = df_ratio.append(pd.merge(df_ha, df_hd, on=['n_e', 'T_e'], 
                                    suffixes=('_1', '_2')))
# calculate the ratio between pecs
df_ratio['ratio'] = df_ratio.X_e_1 / df_ratio.X_e_2
# group by wavelength ratio
group_ratio = df_ratio.groupby(['wavelength_1', 'wavelength_2'])

In [194]:
group_b = group_ratio.get_group((ha, hb))
group_g = group_ratio.get_group((ha, hg))
group_d = group_ratio.get_group((ha, hd))

In [195]:
# let's try to plot group d
#fig = plt.figure()
#ax = fig.add_subplot()
#for wv, group_T in zip(['r_beta', 'r_gamma', 'r_delta'],[group_b, group_g, group_d]):
group_d_T = group_g[group_g.T_e < 20].groupby('n_e')
for name, group in group_d_T:
    if 1e10 <= float(name) < 1e12:
        plt.plot(group.T_e, np.log10(group.ratio), 
            label='n = {:.3e}'.format(float(name)))
        plt.xlabel('T (eV)')
        plt.ylabel('log(ratio)')
        plt.title('$H_{\\alpha} / H_{\\gamma}$')
        plt.legend()
        #plt.savefig('r_gamma.svg', format='svg')

In [204]:
# let's try to plot group d but with respect to densities
#fig = plt.figure()
#ax = fig.add_subplot()
group_d_n = group_d[(group_d.n_e < 1e15) & (group_d.n_e > 1e7)].groupby('T_e')
for name, group in group_d_n:
    if 1 < float(name) < 20:
        plt.plot(np.log10(group.n_e), (group.X_e_2 / group.X_e_1), 
            label='T = {:.3e}'.format(float(name)))
plt.xlabel('n ($cm^{3}$)')
plt.ylabel('ratio')
plt.legend()

In [56]:
group_g.ix[(group_g.ratio - 0.059).abs().argsort()[:10]]

,n_e,wavelength_1,T_e,X_e_1,wavelength_2,X_e_2,ratio
141,5.000000e+14,4339.9,1.5,1.620000e-14,6561.9,2.330000e-13,0.069528
140,2.000000e+14,4339.9,1.5,1.600000e-14,6561.9,3.860000e-13,0.041451
139,1.000000e+14,4339.9,1.5,1.580000e-14,6561.9,5.660000e-13,0.027915
138,5.000000e+13,4339.9,1.5,1.550000e-14,6561.9,7.930000e-13,0.019546
142,1.000000e+15,4339.9,1.5,1.620000e-14,6561.9,1.620000e-13,0.100000
167,2.000000e+15,4339.9,2,9.690000e-15,6561.9,5.890000e-13,0.016452
137,2.000000e+13,4339.9,1.5,1.480000e-14,6561.9,1.090000e-12,0.013578
136,1.000000e+13,4339.9,1.5,1.410000e-14,6561.9,1.270000e-12,0.011102
166,1.000000e+15,4339.9,2,9.670000e-15,6561.9,9.110000e-13,0.010615
135,5.000000e+12,4339.9,1.5,1.320000e-14,6561.9,1.390000e-12,0.009496


In [61]:
group_b.ix[(group_b.ratio - 4).abs().argsort()[:50]]

,n_e,wavelength_1,T_e,X_e_1,wavelength_2,X_e_2,ratio
292,1.000000e+09,6561.9,20,2.030000e-09,4860.6,5.070000e-10,4.003945
272,2.000000e+10,6561.9,15,1.630000e-09,4860.6,4.090000e-10,3.985330
293,2.000000e+09,6561.9,20,2.020000e-09,4860.6,5.030000e-10,4.015905
291,5.000000e+08,6561.9,20,2.030000e-09,4860.6,5.100000e-10,3.980392
289,1.000000e+08,6561.9,20,2.050000e-09,4860.6,5.160000e-10,3.972868
249,5.000000e+10,6561.9,10,1.120000e-09,4860.6,2.780000e-10,4.028777
290,2.000000e+08,6561.9,20,2.040000e-09,4860.6,5.140000e-10,3.968872
294,5.000000e+09,6561.9,20,2.000000e-09,4860.6,4.960000e-10,4.032258
288,5.000000e+07,6561.9,20,2.050000e-09,4860.6,5.170000e-10,3.965184
248,2.000000e+10,6561.9,10,1.130000e-09,4860.6,2.850000e-10,3.964912


In [94]:
d = group_g.ix[(group_g.ratio - 1/0.05948).abs().argsort()[:5]]
d

,n_e,wavelength_1,T_e,X_e_1,wavelength_2,X_e_2,ratio
141,5.000000e+14,6561.9,1.5,2.330000e-13,4339.9,1.620000e-14,14.382716
142,1.000000e+15,6561.9,1.5,1.620000e-13,4339.9,1.620000e-14,10.000000
140,2.000000e+14,6561.9,1.5,3.860000e-13,4339.9,1.600000e-14,24.125000
143,2.000000e+15,6561.9,1.5,1.100000e-13,4339.9,1.630000e-14,6.748466
96,5.000000e+07,6561.9,1,3.070000e-14,4339.9,6.580000e-15,4.665653


In [184]:
hbhd = pd.merge(df_hd, df_hb, on=['n_e', 'T_e'], suffixes=('_1', '_2'))
hghd = pd.merge(df_hg, df_hd, on=['n_e', 'T_e'], suffixes=('_1', '_2'))
hbhg = pd.merge(df_hb, df_hg, on=['n_e', 'T_e'], suffixes=('_1', '_2'))

group_g_n = hbhd[(hbhd.n_e < 1e12) & (hbhd.n_e > 1e9)].groupby('T_e')
for name, group in group_d_n:
    if 1 < float(name) < 20:
        plt.plot(np.log10(group.n_e), np.log10(group.X_e_2), 
            label='T = {:.3e}'.format(float(name)))
plt.xlabel('n ($cm^{3}$)')
plt.ylabel('ratio')
plt.legend()

#hbhd['ratio'] = hbhd['X_e_1'] / hbhd['X_e_2']
#hbhd_filter = hbhd.ix[(hbhd.ratio - 6.28).abs().argsort()]
#hbhd_filter[(hbhd_filter.n_e < 1e12) & (hbhd_filter.n_e > 1e9) & (hbhd_filter.T_e > 1) & (hbhd_filter.T_e < 10)]

In [168]:
dd = group_b.ix[(group_b.ratio - 1/0.039).abs().argsort()]
dd = dd[(dd.T_e < 10) & (dd.n_e < 1e12) & (dd.n_e > 1e9) & (dd.T_e >= 1)]
dd.groupby('T_e').mean()

,n_e,X_e_1,X_e_2,ratio
T_e,,,,
1.0,1.108750e+11,3.017500e-14,4.408750e-15,6.850862
1.5,1.108750e+11,1.607500e-12,2.810000e-13,5.728794
2.0,1.108750e+11,1.168750e-11,2.207500e-12,5.304382
3.0,1.108750e+11,8.137500e-11,1.703750e-11,4.786435
5.0,1.108750e+11,3.738750e-10,8.552500e-11,4.382356
7.0,1.108750e+11,5.953750e-10,1.685000e-10,3.541127


In [222]:
simil_fantz_d = group_d[['n_e', 'T_e', 'ratio']]
simil_fantz_d = simil_fantz_d[(simil_fantz_d.T_e < 10) & (simil_fantz_d.T_e > 1) & (simil_fantz_d.n_e > 1e9) & (simil_fantz_d.n_e < 1e11)]
simil_fantz_d = simil_fantz_d.pivot(index='n_e', columns='T_e', values='ratio')
simil_fantz_d = simil_fantz_d.mean()
simil_fantz_d.to_csv('ratio_de.csv', sep='\t', header='T_e')